In [15]:
import pandas as pd
import numpy as np
import glob
import os
import matplotlib.pyplot as plt

# --- Base folder path ---
base_folder_path = "/home/aricept094/mydata/ANOVA/radius4"  # Update this to your base folder path

# --- Folder names and display names ---
folder_names = {
    "casia_less_than_1": "Under 1 D",
    "casia1-2": "1-2 D",
    "casia2-4": "2-4 D",
    "casia_more_than_4": "4-6 D"  # Assuming 4-6D as per description, adjust if needed
}

# --- Color Palette ---
folder_names_colors = {
    "casia_less_than_1": "#E64B35",  # Red
    "casia1-2": "#4DBBD5",           # Blue
    "casia2-4": "#00A087",           # Teal
    "casia_more_than_4": "#3C5488"   # Navy
}

# --- Radii of interest ---
radii_of_interest = [4, 8, 12, 16, 20, 24]

# --- Angle Labels ---
angle_ticks_degrees = [0, 90, 180, 270, 360]
angle_labels_text = ['Temporal', 'Superior', 'Nasal', 'Inferior', 'Temporal']

# --- Create output directory if it doesn't exist ---
output_dir = "pachymetry_plots"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# --- Loop through each radius ---
for radius_of_interest in radii_of_interest:
    fig, ax = plt.subplots(figsize=(12, 6))  # Get figure and axes object
    
    # --- Loop through each folder ---
    for folder_name, display_name in folder_names.items():
        folder_path = os.path.join(base_folder_path, folder_name)
        all_files = glob.glob(os.path.join(folder_path, "*.csv"))
        all_dfs = []
        
        for f in all_files:
            df = pd.read_csv(f)
            all_dfs.append(df)
            
        if not all_dfs:
            print(f"No CSV files found in folder: {folder_name}")
            continue  # Skip to the next folder if no files are found
            
        combined_df = pd.concat(all_dfs, ignore_index=True)
        
        # Filter data for the current radius
        radius_df = combined_df[combined_df['Radial_Index'] == radius_of_interest].copy()
        
        # --- Calculate regular mean for each meridian at the current radius ---
        if not radius_df.empty:
            mean_values = radius_df.groupby('Meridian_Index')['Pachymetry_Value'].agg(['mean']).reset_index()
            mean_values = pd.merge(mean_values, radius_df[['Meridian_Index', 'Meridian_Angle_Deg']].drop_duplicates(subset=['Meridian_Index']), on='Meridian_Index', how='left')
            mean_values.rename(columns={'mean': 'Regular Mean'}, inplace=True)
            
            # Plot Regular Mean for the current folder using Meridian_Angle_Deg for x-axis
            ax.plot(mean_values['Meridian_Angle_Deg'], mean_values['Regular Mean'],
                   label=display_name,
                   linestyle='-', marker='o', markersize=3,
                   color=folder_names_colors[folder_name])
        else:
            print(f"No data found for Radial_Index = {radius_of_interest} in folder {folder_name}")

    # --- Customize plot for the current radius ---
    ax.set_title(f'Pachymetry Mean at Radius {radius_of_interest}')
    ax.set_xlabel('Meridian Angle')
    ax.set_ylabel('Pachymetry (µm)')
    
    # Set up the primary x-axis for degrees
    ax.set_xticks(angle_ticks_degrees)
    ax.set_xticklabels(angle_ticks_degrees)
    ax.set_xlim([-10, 370])
    
    # Adjust the bottom margin to make room for the second set of labels
    plt.subplots_adjust(bottom=0.15)
    
    # Create a twin axis for the anatomical labels
    ax2 = ax.twiny()
    ax2.spines['top'].set_visible(False)
    
    # Set the position of ax2 to bottom
    ax2.xaxis.set_ticks_position('bottom')
    ax2.xaxis.set_label_position('bottom')
    
    # Move ax2 below ax1
    ax2.spines['bottom'].set_position(('outward', 40))
    
    # Set up the secondary x-axis for anatomical labels
    ax2.set_xlim(ax.get_xlim())
    ax2.set_xticks(angle_ticks_degrees)
    ax2.set_xticklabels(angle_labels_text)
    
    ax.grid(True, linestyle='--', alpha=0.6)
    
    # Add legend with title
    ax.legend(title='Astigmatism Group')
    
    plt.tight_layout()
    
    # Save the figure with 550 DPI
    output_filename = os.path.join(output_dir, f'pachymetry_radius_{radius_of_interest}.png')
    plt.savefig(output_filename, dpi=550, bbox_inches='tight')
    plt.close()  # Close the figure to free up memory

print("Plots generated and saved in the 'pachymetry_plots' directory.")

Plots generated and saved in the 'pachymetry_plots' directory.


In [19]:
import os
from PIL import Image

def combine_images_grid(image_folder, output_filename="combined_image_grid.png", dpi=550, spacing_horizontal=10, spacing_vertical=10, images_per_row=2, max_rows=4):
    """
    Combines PNG images from a folder into a grid (specified rows and columns) image,
    adding horizontal and vertical spacing between images, and sets DPI.

    Args:
        image_folder (str): Path to the folder containing PNG images.
        output_filename (str): Name for the combined output image file.
        dpi (int): DPI (dots per inch) for the output image.
        spacing_horizontal (int): Horizontal spacing in pixels between images.
        spacing_vertical (int): Vertical spacing in pixels between rows of images.
        images_per_row (int): Number of images to place in each row (e.g., 2 for 2 columns).
        max_rows (int): Maximum number of rows for the grid (e.g., 4 for up to 4 rows).
    """
    image_files = [f for f in os.listdir(image_folder) if f.lower().endswith('.png')]
    image_files.sort()

    if not image_files:
        print(f"No PNG images found in folder: {image_folder}")
        return

    num_images = len(image_files)
    num_rows_calculated = (num_images + images_per_row - 1) // images_per_row
    num_rows = min(max_rows, num_rows_calculated) # Ensure not exceeding max_rows, but use calculated rows if fewer are needed.


    images = []
    max_width_per_row = [0] * num_rows # Keep track of max width for each row
    max_height_per_row = [0] * num_rows # Keep track of max height for each row
    total_height = 0
    total_width = 0 # Initialize total width, will be max of all row widths

    for index, file in enumerate(image_files):
        try:
            img_path = os.path.join(image_folder, file)
            img = Image.open(img_path)
            images.append(img)
            row_index = index // images_per_row
            if row_index < num_rows: # Only process up to max_rows
                max_height_per_row[row_index] = max(max_height_per_row[row_index], img.height)
                max_width_per_row[row_index] += img.width

        except FileNotFoundError:
            print(f"Error: Image file not found: {img_path}")
            return
        except Image.UnidentifiedImageError:
            print(f"Error: Cannot open or identify image file: {img_path}. Please ensure it is a valid PNG image.")
            return
        except Exception as e:
            print(f"An unexpected error occurred while processing {file}: {e}")
            return

    # Calculate total height and width of combined image
    total_height = sum(max_height_per_row) + spacing_vertical * (num_rows - 1) if num_rows > 1 else sum(max_height_per_row)
    for row_index in range(num_rows):
        num_images_in_row = min(images_per_row, num_images - row_index * images_per_row) # Images in last row might be less
        row_width = sum(images[i].width for i in range(row_index * images_per_row, min((row_index + 1) * images_per_row, num_images)))
        row_width += spacing_horizontal * (num_images_in_row - 1) if num_images_in_row > 1 else 0
        total_width = max(total_width, row_width) # Total width is max of all row widths


    combined_image = Image.new('RGB', (total_width, total_height), color='white') # White background
    x_offset = 0
    y_offset = 0
    image_index = 0

    for row_index in range(num_rows):
        x_offset = 0 # Reset x_offset for each row
        for col_index in range(images_per_row):
            if image_index < num_images: # Ensure we don't go out of bounds
                img = images[image_index]
                combined_image.paste(img, (x_offset, y_offset))
                x_offset += img.width + spacing_horizontal
                image_index += 1
            else:
                break # No more images for this row or grid
        y_offset += max_height_per_row[row_index] + spacing_vertical


    try:
        combined_image.save(output_filename, dpi=(dpi, dpi)) # Set DPI
        print(f"Successfully combined images into a {num_rows}x{images_per_row} grid and saved to {output_filename} at {dpi} DPI")
    except Exception as e:
        print(f"Error saving the combined grid image: {e}")


if __name__ == "__main__":
    image_directory = "/home/aricept094/python/pachymetry_plots"  # Replace with your actual folder path
    output_file = "combined_pachymetry_grid_4row_2col_plot.png"
    output_dpi = 550
    images_in_row = 2 # Now 2 columns
    max_grid_rows = 4 # Maximum 4 rows if needed, but will adjust based on image count.

    combine_images_grid(image_directory, output_file, dpi=output_dpi, images_per_row=images_in_row, max_rows=max_grid_rows)

Successfully combined images into a 3x2 grid and saved to combined_pachymetry_grid_4row_2col_plot.png at 550 DPI
